In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random

import cv2
from PIL import Image

from focus.Focuser import Focuser
from picamera2 import Picamera2, Preview
# pip install -U pymodbus pyserial
from pymodbus.client import ModbusSerialClient

import uuid
import time
from pathlib import Path
from datetime import datetime

from tqdm.auto import tqdm

In [ ]:
def generate_uuid():
    return str(uuid.uuid4())

def generate_datetime():
    return datetime.fromtimestamp(time.time()).strftime(f'%Y%m%d_%H%M%S_%f')

def generate_cache_file(filename, caches_dir='caches', child_dir=None, filetype='', suffix='', prefix=''):
    if child_dir is not None:
        cache = Path(caches_dir) / Path(child_dir)
    else:
        cache = Path(caches_dir)
    cache.mkdir(parents=True, exist_ok=True)
    return cache / Path(
        f"{prefix}{'_' if prefix != '' else ''}{filename}{'_' if suffix != '' else ''}{suffix}{'.' if filetype != '' else ''}{filetype}")

def generate_random_cache_file(caches_dir='caches', child_dir=None, filetype='', suffix='', prefix=''):
    return generate_cache_file(generate_uuid(), caches_dir, child_dir, filetype, suffix, prefix)

def generate_timedate_cache_file(caches_dir='caches', child_dir=None, filetype='', suffix='', prefix=''):
    return generate_cache_file(generate_datetime(), caches_dir, child_dir, filetype, suffix, prefix)

Generate fake data

In [ ]:
for i in range(10):
    print(np.random.randint(1, 4, 12))

## Parameters

In [ ]:
caches_dir = 'datasets/xxx'

JY_DAM0404D_device = '/dev/ttyUSB0'
JY_DAM0404D_baudrate = 9600
JY_DAM0404D_stopbits = 1
JY_DAM0404D_bytesize = 8
JY_DAM0404D_parity = 'N'
JY_DAM0404D_timeout = 1

camera_focus_device_name = '/dev/v4l-subdev1'
focus_step = 2000

AnalogueGain = 0.0
ExposureTime = 200000
Brightness = 0.0
AwbEnable = False  # auto white balance

## Prepare UV Control

In [ ]:
JY_DAM0404D_slave_id = 0x01
JY_DAM0404D_client = ModbusSerialClient(
    port=JY_DAM0404D_device,
    baudrate=JY_DAM0404D_baudrate,
    stopbits=JY_DAM0404D_stopbits,
    bytesize=JY_DAM0404D_bytesize,
    parity=JY_DAM0404D_parity,
    timeout=1
)
JY_DAM0404D_client.connect()

In [ ]:
def obtain_JY_DAM0404D_status():
    reg_addr = 0x03E8
    reg_count = 0x14
    return JY_DAM0404D_client.read_input_registers(reg_addr, reg_count, JY_DAM0404D_slave_id)

def control_JY_DAM0404D_DO(num:int, value:bool):
    reg_addr = 0x0000 + num
    return JY_DAM0404D_client.write_coil(num, value, JY_DAM0404D_slave_id)

In [ ]:
ret = control_JY_DAM0404D_DO(0x00, True)
time.sleep(1.0)
ret = control_JY_DAM0404D_DO(0x00, False)

## Adjust Camera Focus

In [ ]:
focuser = Focuser(camera_focus_device_name)

In [ ]:
focuser.set(Focuser.OPT_FOCUS, focus_step)

## Prepare Camera

In [ ]:
picam2 = Picamera2()
picam2.start_preview(Preview.NULL)
capture_config = picam2.create_still_configuration()
picam2.stop()
picam2.configure(capture_config)
with picam2.controls as ctrl:
    ctrl.AnalogueGain = AnalogueGain
    ctrl.ExposureTime = ExposureTime
    ctrl.Brightness = Brightness
    ctrl.AwbEnable = AwbEnable      # auto white balance
# picam2.set_controls({"AwbEnable": 0})
picam2.start()

In [ ]:
picam2.is_open

In [ ]:
picam2.start()

## Sample Image

In [ ]:
def sample_image(n_img=1, save_path=None):
    img = None
    if n_img == 1:
        img = np.uint8(picam2.capture_array())
    else:
        sumv = None
        for i in range(n_img):
            if sumv is None:
                sumv = np.longdouble(picam2.capture_array())
            else:
                sumv += np.longdouble(picam2.capture_array())
        img = np.uint8(sumv / n_img)

    img_saving = Image.fromarray(img)
    img_saving.save('./survey.jpg')
    if save_path is not None:
        img_saving.save(save_path)
    return img

def sample_image_uv(n_img=1, save_path=None):
    control_JY_DAM0404D_DO(0, True)
    time.sleep(0.5)
    img = sample_image(n_img, save_path)
    control_JY_DAM0404D_DO(0, False)
    time.sleep(0.5)
    return img

Image Glance

In [ ]:
img = sample_image_uv()
figsize=(10, 10)
plt.figure(figsize=figsize)
plt.imshow(img)

Sample One Frame

In [ ]:
child_dir = 'dataset'
suffix = 'blank_ps_2'

In [ ]:
f_name = generate_timedate_cache_file(
    caches_dir=caches_dir,
    child_dir=child_dir,
    filetype='jpg',
    suffix=suffix
)
_ = sample_image_uv(save_path=f_name)
print(f'file has been saved to {f_name}')

Sample Timer

In [ ]:
# In Second
Total_Samples = 30
Set_Timer = 60

child_dir = 'dataset'
suffix = 'sample6'

In [ ]:
for idx in tqdm(range(Total_Samples), desc=' Sample Cycle ', position=0):
    f_name = generate_timedate_cache_file(
        caches_dir=caches_dir,
        child_dir=child_dir,
        filetype='jpg',
        suffix=suffix+f'_{idx}'
    )
    time.sleep(Set_Timer)
    _ = sample_image_uv(save_path=f_name)

## Close Devices

In [ ]:
picam2.close()

In [ ]:
JY_DAM0404D_client.close()